In [8]:
import numpy as np
import pandas as pd
import cvxpy as cp
import math

In [21]:
def phi(n,t):
    return np.array([[4 - 3*math.cos(n*t), 0, 0, (1/n) * math.sin(n*t), (2/n) * (1 - math.cos(n*t)),0],
                 [6*(math.sin(n*t)-n*t),1, 0, -1*(2/n)*( 1 - math.cos(n*t)), (1/n)*(4*math.sin(n*t) - 3*n*t),0],
                 [0,0,math.cos(n*t),0,0,(1/n)*math.sin(n*t)],
                 [3*n*math.sin(n*t),0,0,math.cos(n*t),2*math.sin(n*t),0],
                 [-6*n*(1-math.cos(n*t)),0,0,-2*math.sin(n*t), 4*math.cos(n*t)-3, 0], 
                 [0,0,-1*n*math.sin(n*t), 0, 0, math.cos(n*t)]])

def move(u):
    delta = 1
    n = (2*np.pi) / 96400
    k = phi(n,0)@x_0
    for i in range(1,total):
        k = k + phi(n,i) @ np.array([[0],[0],[0],[u[i][0]*delta],[u[i][1]*delta],[u[i][2]*delta]])
    return np.sum(k**2)

In [22]:
# inital x
x_0 = np.array([0,0,0,20,40,10]).reshape((6,1))

total = 10
m = 1000

In [23]:
x = cp.Variable((total,3))
objective = cp.Minimize(move(x))
constraints = []

for i in range(total):
    constraints += [x[i][0]**2 + x[i][1]**2 + x[i][2]**2 <= m]

prob = cp.Problem(objective, constraints)
sol = prob.solve()

print("Final Distance") 
print(sol)

print("Thrust Vectors at Time Step")
print(x.value)

Final Distance
1.093970388626209e-12
Thrust Vectors at Time Step
[[  0.           0.           0.        ]
 [ -9.94195896 -16.43127596  -5.04297533]
 [ -7.15082023 -15.25506898  -3.56079639]
 [ -5.10648005 -11.82946313  -2.52023541]
 [ -3.36420748  -8.18431679  -1.65056682]
 [ -1.77023922  -4.53150623  -0.86309179]
 [ -0.25412739  -0.91195432  -0.11791567]
 [  1.19251858   2.57629914   0.59242231]
 [  2.55825156   5.82107755   1.26489955]
 [  3.8370613    8.74620868   1.89826043]]
